<a href="https://colab.research.google.com/github/ayselinaydogdu/opencv_goruntu_isleme/blob/main/digit_detection_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Bu projede, el yazısı rakamları tanımak için makine öğrenmesi kullanıldı. CSV dosyasından alınan verilerle Random Forest modeli eğitildi ve kaydedildi. Daha sonra farklı rakam görselleri OpenCV ile işlenip modele uygun hale getirildi. İşlenen görseller modele verilerek hangi rakam olduğu tahmin edildi.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import joblib
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from sklearn.metrics import accuracy_score

In [ ]:
df = pd.read_csv('train.csv')
df.head(3)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB


In [ ]:
eksik_sayisi = df.isnull().sum()
print("Eksik Değerler: \n", eksik_sayisi)

Eksik Değerler: 
 label       0
pixel0      0
pixel1      0
pixel2      0
pixel3      0
           ..
pixel779    0
pixel780    0
pixel781    0
pixel782    0
pixel783    0
Length: 785, dtype: int64


In [ ]:
y = df['label']
x = df.drop('label', axis=1)   # pixel değerleri
x = x / 255.0     # modelin anlaması için pixel değerlerini [0,1] aralığına getirmemiz lazım
# x / 255.0 equals to x_train / 255.0, x_test / 255.0 (hem x_train hem x_test için)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size= 0.2, random_state=42)

In [ ]:
x_train.shape

(33600, 784)

Random Forest Classifier

In [ ]:
rf = RandomForestClassifier(n_estimators=100, max_depth=60, random_state=42)
model = rf.fit(x_train, y_train)
model.score(x_test, y_test)

0.9629761904761904

Modeli Kaydet

In [ ]:
joblib.dump(model, "rakam_model.joblib")

['rakam_model.joblib']

Modeli Yükle

MNIST veri seti, el yazısı ile yazılmış 0–9 arasındaki rakamların (yani 10 sınıf) bulunduğu bir veri setidir. Boyut: 28x28 piksel
Grayscale (siyah-beyaz)
Her piksel değeri: 0 (siyah) ile 255 (beyaz) arasında

In [ ]:
model = joblib.load("rakam_model.joblib")

# Test verisini hazırlıyorsan, dataframe içinde "label" varsa önce ayır
test_labels = df["label"].values
test_pixels = df.drop("label", axis=1).values

# Normalize et (0-255 arası değeri 0-1 aralığına getir)

test_pixels = test_pixels / 255.0    # piksel değerleri [0,255] arasında, modelin anlayacağı hale getir([0,1] arası)
y_pred = model.predict(test_pixels)  # text_pixels içindeki her bir görüntü için rakam tahmini yapılır
for i in range(0,10000):
  print(f"{i+1}. Görüntü - Tahmin: {y_pred[i]}, Gerçek: {test_labels[i]}")

# modelin doğruluğunu hesapla
accuracy = accuracy_score(test_labels, y_pred)
print(f"Model doğruluğu: {accuracy:.4f}")


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


1. Görüntü - Tahmin: 1, Gerçek: 1
2. Görüntü - Tahmin: 0, Gerçek: 0
3. Görüntü - Tahmin: 1, Gerçek: 1
4. Görüntü - Tahmin: 4, Gerçek: 4
5. Görüntü - Tahmin: 0, Gerçek: 0
6. Görüntü - Tahmin: 0, Gerçek: 0
7. Görüntü - Tahmin: 7, Gerçek: 7
8. Görüntü - Tahmin: 3, Gerçek: 3
9. Görüntü - Tahmin: 5, Gerçek: 5
10. Görüntü - Tahmin: 3, Gerçek: 3
11. Görüntü - Tahmin: 8, Gerçek: 8
12. Görüntü - Tahmin: 9, Gerçek: 9
13. Görüntü - Tahmin: 1, Gerçek: 1
14. Görüntü - Tahmin: 3, Gerçek: 3
15. Görüntü - Tahmin: 3, Gerçek: 3
16. Görüntü - Tahmin: 1, Gerçek: 1
17. Görüntü - Tahmin: 2, Gerçek: 2
18. Görüntü - Tahmin: 0, Gerçek: 0
19. Görüntü - Tahmin: 7, Gerçek: 7
20. Görüntü - Tahmin: 5, Gerçek: 5
21. Görüntü - Tahmin: 8, Gerçek: 8
22. Görüntü - Tahmin: 6, Gerçek: 6
23. Görüntü - Tahmin: 2, Gerçek: 2
24. Görüntü - Tahmin: 0, Gerçek: 0
25. Görüntü - Tahmin: 2, Gerçek: 2
26. Görüntü - Tahmin: 3, Gerçek: 3
27. Görüntü - Tahmin: 6, Gerçek: 6
28. Görüntü - Tahmin: 9, Gerçek: 9
29. Görüntü - Tahmin: 9, Gerç

In [ ]:
# 4. Görseli oku
img = cv2.imread("img_26.jpg", cv2.IMREAD_GRAYSCALE)   #görüntünün sadece gri tonlamalı (tek kanal) olarak okunmasını sağlar

img = cv2.resize(img, (28, 28))   # görüntü 28x28 piksel boyutuna getirilir çünkü MNIST veri setindeki tüm görüntüler 28x28 boyutundadır ve modelin eğitildiği veri ile aynı boyutta giriş beklenir.
img = img.flatten().reshape(1, -1) / 255.0   # flatten() fonksiyonu, 28x28 boyutundaki görüntüyü tek boyutlu (1D) hale getirir.

tahmin = model.predict(img)   # görüntünün hangi rakam olduğu tespit edilir
print("Tahmin edilen rakam:", tahmin[0])   # tahmin bir NumPy dizisidir ve içinde bir adet rakam tahmini vardır.


Tahmin edilen rakam: 4


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


# img = img.flatten().reshape(1,-1)
- flatten() fonksiyonu, 28x28 boyutundaki görüntüyü tek boyutlu (1D) hale getirir. (784)

- Bir adet görüntü veriyoruz (örnek sayısı = 1)
Her görüntü 28x28 piksel, yani 784 özellik (her piksel bir özellik)

- Bu yüzden giriş (1, 784) şeklinde olmalı:
1 satır = 1 görüntü (örnek)
784 sütun = her pikselin değeri (özellikler)